In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Generator function

Simply put, a generator is a function that returns an object (an iterator) that we can iterate over (one value at a time). Put another way, generator functions generate a sequence of values over time. It is coded with normal `def` statements but use `yield` statements to return results one at a time, suspending and resuming their **state** between each. The **state** that generator functions retain when they are suspended includes both their **code location**, and their entire **local scope**. When called, generator functions don’t return a result--- they return a result generator that can appear in any iteration context. This is because generators use the iterator protocol where the returned iterator's `__next__` method is called via `next`. Some difference between a generator function and a normal function are:

* Generator functions contain one or more `yield` statements
* When called, generator functions return an object (iterator)
* Methods like `__iter__` and `__next__` are implemented automatically for generators, so we can iterate through the items using `next`
* Generator functions' local variables retain information between results, making them available when the functions are resumed
* Finally, when the function terminates, `StopIteration` is raised automatically on further calls

Because generator functions do not construct a result list all at once, they save memory space and allow computation time to be split across result requests. Generator functions ultimately perform their delayed-results magic by implementing the iteration protocol.

### The yield

The `yield` statement suspends the function and sends a value back to the caller, but retains enough state to enable the function to resume from where it left off. When resumed, the function continues execution immediately **after** the last `yield` run. From the function’s perspective, this allows its code to produce a series of values over time, rather than computing them all at once and sending them back in something like a list. From the caller’s perspective, the generator’s `__next__` method resumes the function and runs until either the next `yield` result is returned or a `StopIteration` is raised.

In [2]:
# Simple generator function
def gensquares(N):
    for i in range(N):
        yield i**2

The generator function above can be used to generate the squares of a series of numbers over time. This function yields a value, and returns to its caller, each time through the loop; when it is resumed, its prior state is restored, including the last values of its variables `i` and `N`, and control picks up again immediately after the `yield` statement.

In [3]:
# Returns an interator
gen = gensquares(5)
gen
# Calling iter() is a no-op since gen is an iterator
iter(gen) is gen

<generator object gensquares at 0x104b883c0>

True

In [4]:
for i in gensquares(5):
    print(i, end=" : ")

0 : 1 : 4 : 9 : 16 : 

When the generator object is used in the body of a `for` loop, the first iteration starts the function and gets its first result; thereafter, control returns to the function after its `yield` statement each time through the loop. To see this, we could step through the iterator manually:

In [5]:
# Manually calling next
gen = gensquares(3)
# No op
I = gen.__iter__()
I is gen

True

In [6]:
next(I)
next(I)
next(I)

0

1

4

In [7]:
try:
    next(I)
except StopIteration as e:
    e

StopIteration()

As can be seen, `for` loops (and other iteration contexts) work with generators by calling the `__next__` method of the returned iterator (`gen` or `I` in our example) repeatedly, until an exception is caught. For a generator, the effect is to produce yielded values over time. If the object to be iterated over does not support this protocol, for loops instead use the indexing protocol to iterate. The generator function above can be replaced with other iteration tools:

In [8]:
# Normal function
def buildsqures(N):
    container = []
    for i in range(N):
        container.append(i**2)
    return container


# Result
for i in buildsqures(5):
    print(i, end=" : ")

0 : 1 : 4 : 9 : 16 : 

In [9]:
# List comprehension
for i in [i**2 for i in range(5)]:
    print(i, end=" : ")

0 : 1 : 4 : 9 : 16 : 

In [10]:
# Map
for i in map((lambda x: x**2), range(5)):
    print(i, end=" : ")

0 : 1 : 4 : 9 : 16 : 

### Send method

In Python 2.5, a `send` method was added to the generator function protocol. The `send` method advances to the next item in the series of results, just like `__next__`, but also provides a way for the caller to communicate with the generator, to affect its operation. Technically, `yield` is now an expression form that returns the item passed to `send`, not a statement (though it can be called either way--- as `yield X`, or `A = (yield X)`). The expression must be enclosed in parentheses unless it’s the only item on the right side of the assignment statement. For example, `X = yield Y` is OK , as is `X = (yield Y) + 42`. When this extra protocol is used, values are sent into a generator `G` by calling `G.send(value)`. The generator’s code is then resumed, and the `yield` expression in the generator returns the value passed to `send`. If the regular `G.__next__()` method (or its `next(G)` equivalent) is called to advance, the `yield` simply returns `None`.

In [11]:
# Generator function
def gen():
    for i in range(10):
        # The name X binds to the yielded value
        X = yield i
        print(X)

In [12]:
G = gen()
# Must call next() first, to start generator
next(G)

0

In [13]:
# Advance, and send value to yield expression, which then gets printed
# Notice that 'i' is still yielded as the elements of range(10)
G.send(12)
G.send(27)

12


1

27


2

In [14]:
# The next() and X.__next__() sends None
next(G)

None


3

The `send` method can be used, for example, to code a generator that its caller can terminate by sending a termination code, or redirect by passing a new position in data being processed inside the generator.

### Other methods

In addition, generators in 2.5 and later also support a `throw(type)` method to raise an exception inside the generator at the latest `yield`, and a `close` method that raises a special `GeneratorExit` exception inside the generator to terminate the iteration entirely.

### Counting spins

 Assume we have the following list of tuples. We want to know how many red spins separate a black spin, on average. We need a function which will yield the count of gaps as it examines the spins. We can then call this function repeatedly to get the gap information.

In [15]:
# Initialize spins
spins = [
    ("red", "18"),
    ("black", "13"),
    ("red", "7"),
    ("red", "5"),
    ("black", "13"),
    ("red", "25"),
    ("red", "9"),
    ("black", "26"),
    ("black", "15"),
    ("black", "20"),
    ("black", "31"),
    ("red", "3"),
]

In [16]:
# Generator function
def count(spins):
    # Initialize red counter
    red_count = 0

    for index in spins:
        # If red, then add one to the counter
        if index[0] == "red":
            red_count += 1
        # If black, then yield the number of red counts
        elif index[0] == "black":
            # This first yield returns control (and the value red_count) to the context that called the for loop (which is the function scope of the count function)
            yield red_count
            # Re-initialize red counter to 0 once a black is reached, each time after the function picks back up after the yield above
            red_count = 0
    # The second yield statement takes the yielded value from the first yield statement, and yields that value to the caller of the count function
    yield red_count

In [17]:
for i in count(spins):
    print(i, end=", ")

1, 2, 2, 0, 0, 0, 1, 

Code logic:

* We initializes `red_count` to count the number of red's before a black is reached in the sequence of tuples. 
  
* Then, we steps through the individual tuples in `spins` in the order presented. For red's, the count is incremented.

* Once a black spin is reached, we `yield` the variable `red_count`, which has so far recorded the number of red's before this most recent occurence of black. 
  
*  When we `yield` a result (the second `yield` statement), the generator produces single result value (`red_count`) while saving all the local variables--- `index` and the location of the `yield` statement--- so that it can be continued from this point.

* When the function is continued, it resumes right after the first `yield` statement in the `ifelse`: the counter will be reset, and the for loop will advance to examine the next tuple (`index`) in the sequence `spins`.

* When the sequence is exhausted, we also `yield` the final count.

### Generating an infinite sequence

In [18]:
# Infinite sequence
def infinite_sequence():
    # Initialize first element of the sequence
    num = 0
    # Infinite loop
    while True:
        # Yield the element
        yield num
        # When the function picks back up, increment the element by 1
        num += 1

In [19]:
# Examine
gen = infinite_sequence()
gen
iter(gen) is gen

<generator object infinite_sequence at 0x104bbf270>

True

In [20]:
gen.__next__()
gen.__next__()
gen.__next__()
gen.__next__()
gen.__next__()

0

1

2

3

4

### Fibonacci

Finding the sum of squares of numbers in the Fibonacci series:

In [21]:
def fibonacci_numbers(nums):
    # Initialize first two numbers
    x, y = 0, 1
    for i in range(nums):
        # The next becomes the current
        # The current plus the next becomes the next
        x, y = y, x + y
        # Yield the current value to the caller
        yield x


def square(nums):
    # Takes the generator object (iterator) returned by fibonacci_numbers(), and square each of its element in a for loop body
    for num in nums:
        yield num**2


print(sum(square(fibonacci_numbers(10))))

4895


Code logic:

* The iterator returned by `fibonacci_numbers()` is passed by reference to `square`.

* Internally, `square` passes the iterator or generator object returned by `fibonacci_numbers()`, and squares each of its element.

* Notice that `square` also returns a generator object, which we passed to `sum`.


### Palindrome

A palindrome is a word, number, phrase, or other sequence of characters which reads the same backward as forward, such as madam or racecar.

In [22]:
# Predicate function for checking if an input number is a palindrome
def is_palindrome(num):
    # If num < 10, then num // 10 == 0 is True
    # // is floordivision
    if num // 10 == 0:
        return False
    # Initialize
    temp = num
    reversed_num = 0

    while temp != 0:
        # Modulo %
        reversed_num = (reversed_num * 10) + (temp % 10)
        temp = temp // 10

    if num == reversed_num:
        return True
    else:
        return False

In [23]:
# Generator function for palindromes
def infinite_palindromes():
    num = 0
    while True:
        if is_palindrome(num):
            # Yield the value to the context that called the while loop and bind the name 'i' to this value
            i = yield num
            # After the function resumes, check if i is None
            # Essentially, we ask: did the caller send back any value via 'send' or was 'next' used to advance instead
            if i is not None:
                num = i
        # Increment
        num += 1

Code logic:

* We set up the infinite loop with `while`.

* As of Python 2.5, `yield` is an expression, rather than a statement. This allows us to manipulate the yielded value, where `i` takes the value that is `yield`ed. This also allows us to `send()` a value back to the generator, so, when execution picks up after `yield`, the variable `i` will take the value that is sent back.

* After the function resumes, we check if `i` is not None, which could happen if `next()` is called on the generator object. If `i` has a value (`i is not None == True`), then we update num with the new value `i` that we sent back using the `send()` method. But regardless of whether or not i holds a value, we will increment num and start the loop again.

In [24]:
# Generator object
gen = infinite_palindromes()
gen.__next__()
gen.__next__()
gen.__next__()
gen.__next__()
gen.__next__()

11

22

33

44

55

In [25]:
for i in gen:
    digits = len(str(i))
    gen.send(10 ** (digits))
    if digits > 5:
        break

101

1001

10001

100001

1000001

Code logic:

* The loop only yields a value once a palindrome is found. It uses len() to determine the number of digits in that palindrome. 
  
* Then, it sends `10 ** digits` to the generator. This brings execution back into the generator logic and binds the name `i` to the object `10 ** digits`. Since `i` now has a value, the program updates num, increments, and checks for palindromes again.

* Once the program finds and yields another palindrome, we will iterate via the `for` loop. This is the same as iterating with `next()`. The generator also picks up `i = (yield num)`. If `num` is not a palindrome (`is_palindrome(num) == False`), then `i` is `None`, because the code in the if block will not run and we will not be explicitly sending a value.

What we’ve created here is a coroutine, or a generator function into which you can pass data.

In [26]:
# Using the throw method
gen = infinite_palindromes()
for i in gen:
    print(i)
    digits = len(str(i))
    # Throw an exception if digits gets to 5
    if digits == 5:
        gen.throw(ValueError("Error"))
    gen.send(10 ** (digits))

11


101

111


1001

1111


10001

10101


ValueError: Error

In [27]:
# Using close
gen = infinite_palindromes()
for i in gen:
    print(i)
    digits = len(str(i))
    if digits == 5:
        gen.close()
    gen.send(10 ** (digits))

11


101

111


1001

1111


10001

10101


StopIteration: 